# BA885 Team Project
#### Christian Lawrence, Tianzheng Mao, Tiam Moradi, Phoenix Wang

To access the dataset, please add the following shared folders to your working directory:
* https://tinyurl.com/3cn2zk8u (labeled images)
* https://tinyurl.com/ypaf8wtt (unlabeled images)

# Environment Setup

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL
import PIL.Image
import pathlib
import matplotlib.image as mpimg
from matplotlib import rcParams
from pathlib import Path
import pathlib

# Data Setup

In [54]:
# # load all the labeled images
# data_dir = pathlib.Path('Labeled')
# print('Total number of labeled images:', len(list(data_dir.glob('*/*.jpg'))))

In [55]:
# # shape of each image
# tf.keras.preprocessing.image.img_to_array(PIL.Image.open(str(building[0]))).shape

## Preprocessing

Use 80% of the images for training and 20% for validation.

In [56]:
batch_size = 32
img_height = 150
img_width = 150

In [57]:
# generate training set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=885,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [58]:
# generate validation set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=885,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Each batch contains 32 images of shape `(150, 150, 3)` and their corresponding labels.

In [59]:
# shape of each batch
image_batch, label_batch = next(iter(train_ds))
print(image_batch.shape)
print(label_batch.shape)

(32, 150, 150, 3)
(32,)


## Rescaling

The RGB channel values are in the `[0, 255]` range. We'll rescale the values to be in the `[0, 1]` range.

In [60]:
# value range from 0 to 255
image_batch, label_batch = next(iter(train_ds))
print('Minimum value:', np.min(image_batch))
print('Maximum value:', np.max(image_batch))

Minimum value: 0.0
Maximum value: 255.0


In [61]:
standardization = tf.keras.layers.experimental.preprocessing.Rescaling(1/255)
train_ds = train_ds.map(lambda x, y: (standardization(x), y))
val_ds = val_ds.map(lambda x, y: (standardization(x), y))

In [62]:
# value range from 0 to 1
image_batch, label_batch = next(iter(train_ds))
print('Minimum value:', np.min(image_batch))
print('Maximum value:', np.max(image_batch))

Minimum value: 0.0
Maximum value: 1.0


# Model Building and Tuning

In [63]:
# configure performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [64]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=16, max_value=64, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,8]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=16, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=8, max_value=16, step=2),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=8, max_value=16, step=2),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [65]:
import keras_tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras

In [66]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output',project_name="BA_885")

In [67]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,min_delta=0.0001)
tuner_search.search(train_ds, validation_data=val_ds,epochs=5,callbacks=[stop_early])

Trial 10 Complete [00h 00m 51s]
val_accuracy: 0.17499999701976776

Best val_accuracy So Far: 0.7360000014305115
Total elapsed time: 00h 09m 12s
INFO:tensorflow:Oracle triggered exit


In [68]:
model=tuner_search.get_best_models(num_models=1)[0]

In [69]:
model.build(image_batch.shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (32, 143, 143, 32)        6176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 71, 71, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 69, 69, 16)          4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 34, 34, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 30, 30, 14)          5614      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 15, 15, 14)          0         
_________________________________________________________________
flatten (Flatten)            (32, 3150)                0

In [70]:
epochs = 10
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
439/439 [==============================] - 9s 19ms/step - loss: 0.5555 - accuracy: 0.7999 - val_loss: 0.6872 - val_accuracy: 0.7570
Epoch 2/10
439/439 [==============================] - 8s 19ms/step - loss: 0.5058 - accuracy: 0.8193 - val_loss: 0.7581 - val_accuracy: 0.7450
Epoch 3/10
439/439 [==============================] - 8s 19ms/step - loss: 0.4596 - accuracy: 0.8381 - val_loss: 0.7470 - val_accuracy: 0.7560
Epoch 4/10
439/439 [==============================] - 8s 19ms/step - loss: 0.4289 - accuracy: 0.8490 - val_loss: 0.7900 - val_accuracy: 0.7517
Epoch 5/10
439/439 [==============================] - 8s 19ms/step - loss: 0.3855 - accuracy: 0.8648 - val_loss: 0.9728 - val_accuracy: 0.7250
Epoch 6/10
439/439 [==============================] - 8s 19ms/step - loss: 0.3723 - accuracy: 0.8674 - val_loss: 0.8593 - val_accuracy: 0.7470
Epoch 7/10
439/439 [==============================] - 8s 19ms/step - loss: 0.3281 - accuracy: 0.8839 - val_loss: 0.8414 - val_accuracy: 0.7487